# **Upload and Run the Below Code in Google Collab**

In [1]:
!pip install --upgrade pip
!pip install mediapipe-model-maker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

# **Import the Required Models**

In [3]:
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import gesture_recognizer

import matplotlib.pyplot as plt

# **Add the Downloaded dataset to your google drives `My Drive` Section. Then use Below Code to connect it to this Project**

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
my_folder_path = '/content/drive/MyDrive/sign_language_dataset'

In [41]:
print(my_folder_path)

/content/drive/MyDrive/sign_language_dataset


In [42]:
print(my_folder_path)
labels = []
for i in os.listdir(my_folder_path):
  if os.path.isdir(os.path.join(my_folder_path, i)):
    labels.append(i)
for label in labels:
  print(label + "\n")

/content/drive/MyDrive/sign_language_dataset
.ipynb_checkpoints

1

2

none



In [43]:
print(len(labels))

4


# **From Below Run All the Steps to Train and then Download the Trained Model**

In [44]:
NUM_EXAMPLES = 5

for label in labels:
    label_dir = os.path.join(my_folder_path, label)

    # Check if directory is not a checkpoint folder or empty
    if not label_dir.endswith(".ipynb_checkpoints") and os.path.isdir(label_dir):
        example_filenames = os.listdir(label_dir)

        # Print the filenames for debugging
        print(f"Files in folder {label_dir}:")
        print(example_filenames)

        # Ensure we don't exceed the available number of images
        num_images_to_display = min(len(example_filenames), NUM_EXAMPLES)

        fig, axs = plt.subplots(1, num_images_to_display, figsize=(10, 2))

        for i in range(num_images_to_display):
            img_path = os.path.join(label_dir, example_filenames[i])
            axs[i].imshow(plt.imread(img_path))
            axs[i].get_xaxis().set_visible(False)
            axs[i].get_yaxis().set_visible(False)

        fig.suptitle(f'Showing {num_images_to_display} examples for {label}')

plt.show()

Files in folder /content/drive/MyDrive/sign_language_dataset/1:
['1_frame_173.jpg', '1_frame_192.jpg', '1_frame_193.jpg', '1_frame_194.jpg', '1_frame_198.jpg', '1_frame_191.jpg', '1_frame_200.jpg', '1_frame_190.jpg', '1_frame_187.jpg', '1_frame_185.jpg', '1_frame_186.jpg', '1_frame_189.jpg', '1_frame_196.jpg', '1_frame_195.jpg', '1_frame_199.jpg', '1_frame_197.jpg', '1_frame_181.jpg', '1_frame_183.jpg', '1_frame_188.jpg', '1_frame_184.jpg', '1_frame_182.jpg', '1_frame_180.jpg', '1_frame_176.jpg', '1_frame_177.jpg', '1_frame_175.jpg', '1_frame_179.jpg', '1_frame_172.jpg', '1_frame_171.jpg', '1_frame_178.jpg', '1_frame_174.jpg', '1_frame_123.jpg', '1_frame_118.jpg', '1_frame_117.jpg', '1_frame_211.jpg', '1_frame_212.jpg', '1_frame_218.jpg', '1_frame_214.jpg', '1_frame_112.jpg', '1_frame_115.jpg', '1_frame_113.jpg', '1_frame_114.jpg', '1_frame_119.jpg', '1_frame_216.jpg', '1_frame_215.jpg', '1_frame_217.jpg', '1_frame_219.jpg', '1_frame_220.jpg', '1_frame_108.jpg', '1_frame_107.jpg', '1_f

ValueError: Number of columns must be a positive integer, not 0

In [45]:
data = gesture_recognizer.Dataset.from_folder(
    dirname=my_folder_path,
    hparams=gesture_recognizer.HandDataPreprocessingParams()
)
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

In [46]:
hparams = gesture_recognizer.HParams(export_dir="exported_model")
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer  [(None, 128)]             0         
 )                                                               
                                                                 
 batch_normalization (Batch  (None, 128)               512       
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 custom_gesture_recognizer_  (None, 5)                 645       
 out (Dense)                                                     
                                                             

In [47]:
loss, acc = model.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")

19/19 [==============================] - 1s 4ms/step - loss: 0.0119 - categorical_accuracy: 1.0000
Test loss:0.01188892312347889, Test accuracy:1.0


In [48]:
model.export_model()
!ls exported_model

Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
best_model_weights.data-00000-of-00001	checkpoint    gesture_recognizer.task  metadata.json
best_model_weights.index		epoch_models  logs


In [49]:
files.download('exported_model/gesture_recognizer.task')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>